In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [9]:
label = LabelEncoder()
df = pd.read_csv('data/car/car.data', header=None)
header = ["buying","maint","doors","persons","lug_boot","safety","class"]


df.columns = header

In [10]:
df

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [14]:
# df['buying'] = label.fit_transform(df['buying']);
# df['maint'] = label.fit_transform(df['maint']);
# df.replace("5more","5", inplace=True)
# df.replace("more","5", inplace=True)
# df = df.astype({'doors':'int','persons':'int'})
# df['lug_boot'] = label.fit_transform(df['lug_boot']);
# df['safety'] = label.fit_transform(df['safety']);
# df['class'] = label.fit_transform(df['class']);

for column in df.columns:
    df[column] = label.fit_transform(df[column])

In [15]:
df

,buying,maint,doors,persons,lug_boot,safety,class
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2
...,...,...,...,...,...,...,...
1723,1,1,3,2,1,2,1
1724,1,1,3,2,1,0,3
1725,1,1,3,2,0,1,2
1726,1,1,3,2,0,2,1


In [16]:
df['class'].value_counts()

# needs to solve unbalanced problem

2    1210
0     384
1      69
3      65
Name: class, dtype: int64

In [62]:
x = df
x = x.drop(columns=['class'], axis=1)

y = df['class']

In [74]:
x

,buying,maint,doors,persons,lug_boot,safety
0,3,3,0,0,2,1
1,3,3,0,0,2,2
2,3,3,0,0,2,0
3,3,3,0,0,1,1
4,3,3,0,0,1,2
...,...,...,...,...,...,...
1723,1,1,3,2,1,2
1724,1,1,3,2,1,0
1725,1,1,3,2,0,1
1726,1,1,3,2,0,2


In [78]:
x.shape

(1728, 6)

In [75]:
y

0       2
1       2
2       2
3       2
4       2
       ..
1723    1
1724    3
1725    2
1726    1
1727    3
Name: class, Length: 1728, dtype: int64

In [79]:
y.shape

(1728,)

In [76]:
x.isna().sum()

buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
dtype: int64

In [77]:
y.isna().sum()

0

## Using SMOTE(Synthetic Minority Over-sampling Technique)

In [35]:
# Using SMOTE(Synthetic Minority Over-sampling Technique)

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto')
x_, y_ = smote.fit_resample(x, y)

In [36]:
x_

,buying,maint,doors,persons,lug_boot,safety
0,3,3,0,0,2,1
1,3,3,0,0,2,2
2,3,3,0,0,2,0
3,3,3,0,0,1,1
4,3,3,0,0,1,2
...,...,...,...,...,...,...
4835,1,2,2,2,1,0
4836,1,0,2,1,1,0
4837,1,1,2,1,1,0
4838,1,1,3,1,0,0


In [37]:
y_

0       2
1       2
2       2
3       2
4       2
       ..
4835    3
4836    3
4837    3
4838    3
4839    3
Name: class, Length: 4840, dtype: int64

In [38]:
y_.value_counts()

2    1210
0    1210
3    1210
1    1210
Name: class, dtype: int64

In [39]:
x_.isna().sum()

buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
dtype: int64

In [48]:
y_.isna().sum()

0

In [49]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.25, random_state=0)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=0)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3630, 6), (1210, 6), (3630,), (1210,))

In [50]:
# SVM

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

svm = SVC(random_state=0)
svm.fit(x_train, y_train)

pred_svm = svm.predict(x_test)

print("\n--- SVM Classifier ---")
print(accuracy_score(y_test, pred_svm))
print(confusion_matrix(y_test, pred_svm))


--- SVM Classifier ---
0.9752066115702479
[[263   4   0   0]
 [  0 296   0   0]
 [ 26   0 298   0]
 [  0   0   0 323]]


In [51]:
# LR

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0)
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)

print ("\n--- Logistic Regression Classifier ---")
print (accuracy_score(y_test, pred_lr))
print (confusion_matrix(y_test, pred_lr))


--- Logistic Regression Classifier ---
0.5595041322314049
[[ 54  67  79  67]
 [131 116  49   0]
 [ 61  37 202  24]
 [ 16   0   2 305]]


In [52]:
# DT

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)
dt.fit(x_train, y_train)

pred_dt = dt.predict(x_test)

print ("\n--- Decision Tree Classifier ---")
print (accuracy_score(y_test, pred_dt))
print (confusion_matrix(y_test, pred_dt))


--- Decision Tree Classifier ---
0.9942148760330578
[[262   1   4   0]
 [  0 296   0   0]
 [  2   0 322   0]
 [  0   0   0 323]]


In [53]:
# RF

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

print ("\n--- Random Forest ---")
rf = RandomForestClassifier(random_state=0)
rf.fit(x_train, y_train)
pred = rf.predict(x_test)
print(accuracy_score(y_test,pred))
print (confusion_matrix(y_test, pred))


--- Random Forest ---
0.9950413223140496
[[266   1   0   0]
 [  0 296   0   0]
 [  5   0 319   0]
 [  0   0   0 323]]


In [54]:
#RF

class_weight = 'balanced'
rf = RandomForestClassifier(class_weight=class_weight)
rf.fit(X_train, Y_train)
pred = rf.predict(x_test)
print(accuracy_score(y_test,pred))
print (confusion_matrix(y_test, pred))

0.9942148760330578
[[266   1   0   0]
 [  0 296   0   0]
 [  6   0 318   0]
 [  0   0   0 323]]


## Using under_sampling

In [63]:
from imblearn.under_sampling import RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy='auto')
x_, y_ = undersampler.fit_resample(x, y)

In [64]:
x_

,buying,maint,doors,persons,lug_boot,safety
392,3,1,2,1,1,0
745,0,2,3,1,0,2
551,0,0,0,1,2,0
695,0,2,1,2,2,0
968,2,3,3,2,1,0
...,...,...,...,...,...,...
1700,1,1,2,2,0,0
1715,1,1,3,1,1,0
1718,1,1,3,1,0,0
1724,1,1,3,2,1,0


In [65]:
y_

392     0
745     0
551     0
695     0
968     0
       ..
1700    3
1715    3
1718    3
1724    3
1727    3
Name: class, Length: 260, dtype: int64

In [66]:
y_.value_counts()

0    65
1    65
2    65
3    65
Name: class, dtype: int64

In [72]:
x_.isna().sum()

buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
dtype: int64

In [73]:
y_.isna().sum()

0

In [67]:
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.25, random_state=0)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((195, 6), (65, 6), (195,), (65,))

In [68]:
# SVM

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

svm = SVC(random_state=0)
svm.fit(x_train, y_train)

pred_svm = svm.predict(x_test)

print("\n--- SVM Classifier ---")
print(accuracy_score(y_test, pred_svm))
print(confusion_matrix(y_test, pred_svm))


--- SVM Classifier ---
0.6923076923076923
[[ 8  0  4  2]
 [ 2 15  0  2]
 [ 8  1  8  1]
 [ 0  0  0 14]]


In [69]:
# LR

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0)
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)

print ("\n--- Logistic Regression Classifier ---")
print (accuracy_score(y_test, pred_lr))
print (confusion_matrix(y_test, pred_lr))


--- Logistic Regression Classifier ---
0.38461538461538464
[[ 5  2  2  5]
 [11  0  6  2]
 [ 6  1  7  4]
 [ 0  0  1 13]]


In [70]:
# DT

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)
dt.fit(x_train, y_train)

pred_dt = dt.predict(x_test)

print ("\n--- Decision Tree Classifier ---")
print (accuracy_score(y_test, pred_dt))
print (confusion_matrix(y_test, pred_dt))


--- Decision Tree Classifier ---
0.8307692307692308
[[10  1  1  2]
 [ 3 14  0  2]
 [ 2  0 16  0]
 [ 0  0  0 14]]


In [71]:
# RT

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

print ("\n--- Random Forest ---")
rf = RandomForestClassifier(random_state=0)
rf.fit(x_train, y_train)
pred = rf.predict(x_test)
print(accuracy_score(y_test,pred))
print (confusion_matrix(y_test, pred))


--- Random Forest ---
0.8
[[ 9  1  2  2]
 [ 1 16  0  2]
 [ 5  0 13  0]
 [ 0  0  0 14]]
